Загружаем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

Читаем исходные файлы

In [2]:
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
Y_train = pd.read_csv('y_train.csv')

f_columns = X_train.columns.difference(['id'])

Производим one hot encoding над всеми признаками.

In [3]:
ohe = OneHotEncoder()
X_all = ohe.fit_transform(
            pd.concat(
                [
                    X_train[f_columns],
                    X_test[f_columns]
                ]))

X_train_ohe = X_all[:X_train.shape[0]]
X_test_ohe = X_all[X_train.shape[0]:]

Перебираем по сетке параметры для логистической регрессии

In [4]:
parameters = {'C': np.logspace(-5, 10, 50)}
clf = GridSearchCV(LogisticRegression(), parameters, scoring='roc_auc', refit=True, cv=5, n_jobs=-1)
clf.fit(X_train_ohe, Y_train['label'])
print(clf.best_score_)

0.832160936686


Записываем в файл результаты предсказания

In [5]:
X_test['prediction'] = clf.predict_proba(X_test_ohe)[:, 1]
X_test[['id', 'prediction']].to_csv('baseline.csv', index=None)